<a href="https://colab.research.google.com/github/reallylongaddress/abnb_preprocessor/blob/master/notebooks/re_price_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup, Tag
import pandas as pd
from google.colab import drive


In [ ]:

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

# Scrape URL: 
provinces_url = 'https://www.immobiliare.it/en/mercato-immobiliare/'

# References
#https://hackersandslackers.com/scraping-urls-with-beautifulsoup

In [ ]:
req = requests.get(provinces_url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())

# Load Regions

In [ ]:
#look for 'table' class: nd-table nd-table--borderBottom
region_table = soup.find_all("table", class_="nd-table nd-table--borderBottom")

#look for the 'tr' class: nd-table__row
regions = []
for regions_tag in soup.find_all("tr", class_="nd-table__row"):
    
    region = ''
    url = ''
    sale = 0
    rent = 0

    for regions_tag_child in regions_tag.children:
        if type(regions_tag_child) == Tag:
            if regions_tag_child.get("data-text") == "(€/m²)":
                link = regions_tag_child.find('a')
                if (link):
                    url = link.get('href')
                    region_name = link.text
            elif regions_tag_child.get("data-text") == "Sale":
                sale = regions_tag_child.text.replace('.', '')
            elif regions_tag_child.get("data-text") == "Rent":
                rent = regions_tag_child.text.replace(',', '.')
                
    regions.append([region_name, sale, rent, url])

regions_df = pd.DataFrame(data = regions, columns=['region', 'sale_mean', 'rent_mean', 'url'])
regions_df['sale_mean'] = regions_df['sale_mean'].astype(int)
regions_df['rent_mean'] = regions_df['rent_mean'].astype(float)
regions_df

,region,sale_mean,rent_mean,url
0,Abruzzo,1293,7.97,https://www.immobiliare.it/en/mercato-immobili...
1,Basilicata,1375,7.13,https://www.immobiliare.it/en/mercato-immobili...
2,Calabria,920,7.12,https://www.immobiliare.it/en/mercato-immobili...
3,Campania,1881,9.33,https://www.immobiliare.it/en/mercato-immobili...
4,Emilia Romagna,1839,12.19,https://www.immobiliare.it/en/mercato-immobili...
5,Friuli Venezia Giulia,1465,8.94,https://www.immobiliare.it/en/mercato-immobili...
6,Lazio,2465,13.54,https://www.immobiliare.it/en/mercato-immobili...
7,Liguria,2515,10.26,https://www.immobiliare.it/en/mercato-immobili...
8,Lombardia,2174,15.46,https://www.immobiliare.it/en/mercato-immobili...
9,Marche,1517,9.32,https://www.immobiliare.it/en/mercato-immobili...


In [ ]:
# for index, row in regions_df.iterrows():
#     print(f'{index}::{round((((row["rent"]*12)/row["sale"])*100),1)}::{row["region"]}')

In [ ]:
# for index, row in regions_df.iterrows():
#     print(f'{index} {row["sale"]}/{round(row["rent"]*12, 0)}     {round(((row["sale"]/(row["rent"]*12))*100),1)}::{row["region"]}')
    
#     # Median Home Price / Median Annual Rent = Price to Rent Ratio.

# Load Region Detail and Provinces

In [ ]:
regions_data = []
provinces_data = []
# print(f'shape: {regions_df.shape}')
for index, region in regions_df.iterrows():
    print(f'{index}: {region["url"]}')
    
    region_req = requests.get(region["url"], headers)
    region_soup = BeautifulSoup(region_req.content, 'html.parser')    
    
    #load the region price data
    
    region_prices = region_soup.find_all("p", class_="nd-cgHighlighted__subtext")
    
    #SALE 
    #break the string into words
    region_sale_price_parts = region_prices[0].contents[0].split()
    
    #keep only the parts we want
    region_sale_price_parts = [region_sale_price_parts[i] for i in [1,4]]

    #remove '.' thousands separator
    region_sale_price_parts = [price.replace('.', '') for price in region_sale_price_parts]
    # print(f'Sale: {region_sale_price_parts}')
    
    #RENT 
    #break the string into words
    region_rent_price_parts = region_prices[1].contents[0].split()
    
    #keep only the parts we want
    region_rent_price_parts = [region_rent_price_parts[i] for i in [1,4]]

    #remove '.' thousands separator
    region_rent_price_parts = [price.replace(',', '.') for price in region_rent_price_parts]
    
    regions_data.append([index, region_sale_price_parts[0], region_sale_price_parts[1], 
                         region_rent_price_parts[0], region_rent_price_parts[1]])

    #Now load the province data
    province_soup = region_soup.find_all("tr", class_="nd-table__row")

    for province_html in province_soup:
        province_name = ''
        url = ''
        sale = 0
        rent = 0

        for child in province_html.children:
            if type(child) == Tag:
                if child.get("data-text") == "(€/m²)":
                    link = child.find('a')
                    if (link):
                        url = link.get('href')
                        province_name = link.text
                        # print(f'province_name: {province_name}')
                elif child.get("data-text") == "Sale":
                    sale = child.text.replace('.', '')
                elif child.get("data-text") == "Rent":
                    rent = child.text.replace(',', '.')

        provinces_data.append([region['region'], province_name, sale, rent, url])            

#merge region dataframes, drop index, reorder columns
regions_data_df = pd.DataFrame(data = regions_data, columns=['index', 
                                                             'sale_low', 'sale_high', 
                                                             'rent_low', 'rent_high'])
regions_df.reset_index(inplace=True)
regions_df = regions_df.merge(regions_data_df, left_on='index', right_on='index')
# print(regions_df.columns)
regions_df.drop(columns=['index'], inplace=True)
regions_df = regions_df[['region', 'sale_low', 'sale_mean', 'sale_high', 'rent_low', 'rent_mean', 'rent_high', 'url']]

provinces_data_df = pd.DataFrame(data=provinces_data, columns=['region', 'province', 'sale_mean', 
                                                               'rent_mean', 'url'])
                                 
# display(regions_df)
display(provinces_data_df)                                 

0: https://www.immobiliare.it/en/mercato-immobiliare/abruzzo/
1: https://www.immobiliare.it/en/mercato-immobiliare/basilicata/
2: https://www.immobiliare.it/en/mercato-immobiliare/calabria/
3: https://www.immobiliare.it/en/mercato-immobiliare/campania/
4: https://www.immobiliare.it/en/mercato-immobiliare/emilia-romagna/
5: https://www.immobiliare.it/en/mercato-immobiliare/friuli-venezia-giulia/
6: https://www.immobiliare.it/en/mercato-immobiliare/lazio/
7: https://www.immobiliare.it/en/mercato-immobiliare/liguria/
8: https://www.immobiliare.it/en/mercato-immobiliare/lombardia/
9: https://www.immobiliare.it/en/mercato-immobiliare/marche/
10: https://www.immobiliare.it/en/mercato-immobiliare/molise/
11: https://www.immobiliare.it/en/mercato-immobiliare/piemonte/
12: https://www.immobiliare.it/en/mercato-immobiliare/puglia/
13: https://www.immobiliare.it/en/mercato-immobiliare/sardegna/
14: https://www.immobiliare.it/en/mercato-immobiliare/sicilia/
15: https://www.immobiliare.it/en/mercat

,region,province,sale_mean,rent_mean,url
0,Abruzzo,Chieti,1132,6.48,https://www.immobiliare.it/en/mercato-immobili...
1,Abruzzo,L'Aquila,1212,7.57,https://www.immobiliare.it/en/mercato-immobili...
2,Abruzzo,Pescara,1436,8.57,https://www.immobiliare.it/en/mercato-immobili...
3,Abruzzo,Teramo,1378,10.53,https://www.immobiliare.it/en/mercato-immobili...
4,Basilicata,Matera,1540,7.92,https://www.immobiliare.it/en/mercato-immobili...
...,...,...,...,...,...
176,Veneto,Rovigo,966,7.31,https://www.immobiliare.it/en/mercato-immobili...
177,Veneto,Treviso,1550,9.30,https://www.immobiliare.it/en/mercato-immobili...
178,Veneto,Venezia,2326,12.96,https://www.immobiliare.it/en/mercato-immobili...
179,Veneto,Verona,1910,10.12,https://www.immobiliare.it/en/mercato-immobili...


In [ ]:
drive.mount('/drive')


Mounted at /drive


In [ ]:
regions_df.to_csv('/drive/My Drive/Colab_Data/AirBnb/Italy/Sicily/re_regions.csv')

# Load Province Detail and Municipalities

In [ ]:
# provinces_data_df[provinces_data_df['region'] == 'Veneto']


In [ ]:
# abruzzo = provinces_data_df[provinces_data_df['region'] == 'Abruzzo']
# abruzzo

In [ ]:
province_data = []
municipality_data = []

# print(f'shape: {regions_df.shape}')
for index, province in provinces_data_df[provinces_data_df['region'] == "Valle d'Aosta"].iterrows():
# for index, province in provinces_data_df.iterrows():
    print(f'{index}: {province["url"]}')
    
    province_req = requests.get(province["url"], headers)
    province_soup = BeautifulSoup(province_req.content, 'html.parser')    
    
    #load the province price data
    
    province_prices = province_soup.find_all("p", class_="nd-cgHighlighted__subtext")
    
    #SALE 
    #break the string into words
    province_sale_price_parts = province_prices[0].contents[0].split()
    
    #keep only the parts we want
    province_sale_price_parts
    # print(f'province_sale_price_parts: {province_sale_price_parts}')
    if len(province_sale_price_parts) == 6:
      province_sale_price_parts = [province_sale_price_parts[i] for i in [1,4]]

      #remove '.' thousands separator
      province_sale_price_parts = [price.replace('.', '') for price in province_sale_price_parts]
      # print(f'Sale: {province_sale_price_parts}')
      
      #RENT 
      #break the string into words
      province_rent_price_parts = province_prices[1].contents[0].split()
      
      #keep only the parts we want
      province_rent_price_parts = [province_rent_price_parts[i] for i in [1,4]]
      # print(f'Rent: {province_rent_price_parts}')

      #remove '.' thousands separator
      province_rent_price_parts = [price.replace(',', '.') for price in province_rent_price_parts]
      
      # print(index, province_sale_price_parts[0], province_sale_price_parts[1], province_rent_price_parts[0], province_rent_price_parts[1])
      province_data.append([index, province_sale_price_parts[0], province_sale_price_parts[1], 
                          province_rent_price_parts[0], province_rent_price_parts[1]])

    else:
      province_data.append([index, 0,0,0,0])

    #Now load the municipality data
    municipality_soup = province_soup.find_all("tr", class_="nd-table__row")

    for municipality_html in municipality_soup:
        municipality_name = ''
        url = ''
        sale = 0
        rent = 0

        for child in municipality_html.children:
            if type(child) == Tag:
                if child.get("data-text") == "(€/m²)":
                    link = child.find('a')
                    if (link):
                        url = link.get('href')
                        municipality_name = link.text
                        # print(f'municipality_name: {municipality_name}')
                elif child.get("data-text") == "Sale":
                    sale = child.text.replace('.', '')
                elif child.get("data-text") == "Rent":
                    rent = child.text.replace(',', '.')

        municipality_data.append([province['region'], province['province'], municipality_name, sale, rent, url])            
# print(f'municipality_data: {municipality_data}')
municipality_data_df = pd.DataFrame(data=municipality_data, columns=['region', 'province', 'city', 'sale_mean', 'rent_mean', 'url'])

province_data = pd.DataFrame(data=province_data, columns=['index', 'sale_low', 'sale_high', 'rent_low', 'rent_high'])
province_data.set_index('index', inplace=True)
# display(province_data)

#merge province dataframes, reorder columns
provinces_data_df = provinces_data_df.merge(province_data, left_index=True, right_index=True)
display(provinces_data_df)
provinces_data_df = provinces_data_df[['region', 'province', 'sale_low', 'sale_mean', 'sale_high', 'rent_low', 'rent_mean', 'rent_high', 'url']]
# provinces_data_df

100: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/aosta/
101: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/allein/
102: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/antey-saint-andre/
103: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/arnad/
104: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/arvier/
105: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/avise/
106: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/ayas/
107: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/aymavilles/
108: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/bard/
109: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/bionaz/
110: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/brissogne/
111: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/brusson/
112: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-a

,region,province,sale_mean,rent_mean,url,sale_low,sale_high,rent_low,rent_high
100,Valle d'Aosta,Aosta,2008,8.34,https://www.immobiliare.it/en/mercato-immobili...,305,7668,4.50,27.69
101,Valle d'Aosta,Allein,777,7.56,https://www.immobiliare.it/en/mercato-immobili...,305,7668,4.50,27.69
102,Valle d'Aosta,Antey-Saint-Andrè,2196,14.41,https://www.immobiliare.it/en/mercato-immobili...,305,7668,4.50,27.69
103,Valle d'Aosta,Arnad,1244,5.88,https://www.immobiliare.it/en/mercato-immobili...,305,7668,4.50,27.69
104,Valle d'Aosta,Arvier,1409,8.10,https://www.immobiliare.it/en/mercato-immobili...,305,7668,4.50,27.69
...,...,...,...,...,...,...,...,...,...
169,Valle d'Aosta,Valsavarenche,1448,8.14,https://www.immobiliare.it/en/mercato-immobili...,305,7668,4.50,27.69
170,Valle d'Aosta,Valtournenche,3334,18.77,https://www.immobiliare.it/en/mercato-immobili...,305,7668,4.50,27.69
171,Valle d'Aosta,Verrayes,860,4.50,https://www.immobiliare.it/en/mercato-immobili...,305,7668,4.50,27.69
172,Valle d'Aosta,Verrès,1165,6.03,https://www.immobiliare.it/en/mercato-immobili...,305,7668,4.50,27.69


In [ ]:
provinces_data_df.to_csv('/drive/My Drive/Colab_Data/AirBnb/Italy/Sicily/re_provinces.csv')

In [ ]:
# municipality_data_df.head()

# Load Municipality Detail

In [ ]:
municipality_data = []
for index, municipality in municipality_data_df.iterrows():
    print(f'{index}: {municipality["url"]}')
    
    municipality_req = requests.get(municipality["url"], headers)
    municipality_soup = BeautifulSoup(municipality_req.content, 'html.parser')    
    
    #load the province price data
    
    municipality_prices = municipality_soup.find_all("p", class_="nd-cgHighlighted__subtext")
    
    #SALE 
    #break the string into words
    municipality_sale_price_parts = municipality_prices[0].contents[0].split()
    
    #keep only the parts we want
    # print(f'municipality_sale_price_parts: {municipality_sale_price_parts}')
    if len(municipality_sale_price_parts) == 6:
      municipality_sale_price_parts = [municipality_sale_price_parts[i] for i in [1,4]]
      # print(f'municipality_sale_price_parts: {municipality_sale_price_parts}')

      #remove '.' thousands separator
      municipality_sale_price_parts = [price.replace('.', '') for price in municipality_sale_price_parts]
      # print(f'Sale: {municipality_sale_price_parts}')
      
      #RENT 
      #break the string into words
      municipality_rent_price_parts = municipality_prices[1].contents[0].split()
      
      #keep only the parts we want
      municipality_rent_price_parts = [municipality_rent_price_parts[i] for i in [1,4]]
      # print(f'Rent: {municipality_rent_price_parts}')

      #remove '.' thousands separator
      municipality_rent_price_parts = [price.replace(',', '.') for price in municipality_rent_price_parts]
      
      # print(index, province_sale_price_parts[0], province_sale_price_parts[1], province_rent_price_parts[0], province_rent_price_parts[1])
      municipality_data.append([index, municipality_sale_price_parts[0], municipality_sale_price_parts[1], 
                          municipality_rent_price_parts[0], municipality_rent_price_parts[1]])

    else:
      municipality_data.append([index, 0,0,0,0])



Streaming output truncated to the last 5000 lines.
476: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/gressoney-saint-jean/
477: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/hone/
478: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/introd/
479: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/issime/
480: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/issogne/
481: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/jovencan/
482: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/la-magdeleine/
483: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/la-salle/
484: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/la-thuile/
485: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/lillianes/
486: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/montjovet/
487: https://www.immobiliare.it/en/mercato-immobiliare/valle-d-aosta/

In [ ]:
municipality_data = pd.DataFrame(data=municipality_data, columns=['index', 'sale_low', 'sale_high', 'rent_low', 'rent_high'])
municipality_data.set_index('index', inplace=True)
print(f'A: {municipality_data.shape}')
# display(municipality_data2)

#merge province dataframes, reorder columns
municipality_data_df = municipality_data_df.merge(municipality_data, left_index=True, right_index=True)
print(f'A: {municipality_data_df.shape}')
# display(municipality_data_df)
municipality_data_df = municipality_data_df[['region', 'province', 'city', 'sale_low', 'sale_mean', 'sale_high', 'rent_low', 'rent_mean', 'rent_high', 'url']]
municipality_data_df.to_csv('/drive/My Drive/Colab_Data/AirBnb/Italy/Sicily/re_municipalities.csv')

municipality_data_df.head()

A: (5476, 4)
A: (5476, 10)


,region,province,city,sale_low,sale_mean,sale_high,rent_low,rent_mean,rent_high,url
0,Valle d'Aosta,Aosta,Aosta,305,2008,7668,4.50,8.34,27.69,https://www.immobiliare.it/en/mercato-immobili...
1,Valle d'Aosta,Aosta,Allein,305,777,7668,4.50,7.56,27.69,https://www.immobiliare.it/en/mercato-immobili...
2,Valle d'Aosta,Aosta,Antey-Saint-Andrè,305,2196,7668,4.50,14.41,27.69,https://www.immobiliare.it/en/mercato-immobili...
3,Valle d'Aosta,Aosta,Arnad,305,1244,7668,4.50,5.88,27.69,https://www.immobiliare.it/en/mercato-immobili...
4,Valle d'Aosta,Aosta,Arvier,305,1409,7668,4.50,8.10,27.69,https://www.immobiliare.it/en/mercato-immobili...
